## CSE 572: DATA MINING
### HW 3
### K-means Clustering

In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import time

In [8]:
data = pd.read_csv("kmeans_data/data.csv",header = None).to_numpy()
labels = pd.read_csv("kmeans_data/label.csv",header = None).to_numpy()

In [15]:
def euclidean(point1, point2):
    return np.linalg.norm(point1 - point2)

def cosine(point1, point2):
    dot_product = np.dot(point1, point2)
    norm1 = np.linalg.norm(point1)
    norm2 = np.linalg.norm(point2)
    return 1 - (dot_product / (norm1 * norm2))

def jaccard(point1, point2):
    n = np.sum(np.minimum(point1, point2))
    d = np.sum(np.maximum(point1, point2))
    jaccard_score = n / d
    return 1 - jaccard_score

In [107]:
def kmeans(data, k, distance, max_iter=500):
    '''k-means algo'''
    n, m = data.shape
    centroids = data[np.random.choice(n, k, replace=False)]
    clusters = np.zeros(n)
    
    for dummy in range(max_iter):
        for i, point in enumerate(data):
            distances = [distance(point, centroid) for centroid in centroids]
            clusters[i] = np.argmin(distances)
        
        centroids = np.array([data[clusters == j].mean(axis=0) for j in range(k)])
        # if np.all(new_centroids == centroids):
        #     break
        # centroids = new_centroids
        
    sse = sum([distance(data[i], centroids[int(c)])**2 for i, c in enumerate(clusters)])
    
    return clusters, centroids, sse

## Q1

In [109]:
k = len(np.unique(labels)) # 10

iter = 100
print("\nSSE Comparison:")

euclidean_clusters, euclidean_centroids, euclidean_sse = kmeans(data, k, euclidean, iter)
print(f"\nEuclidean:")
print(f"SSE = {euclidean_sse}")
print("\n------------------------------------")

cosine_clusters, cosine_centroids, cosine_sse = kmeans(data, k, cosine, iter)
print(f"\nCosine:")
print(f"SSE = {cosine_sse}")
print("\n------------------------------------")

jaccard_clusters, jaccard_centroids, jaccard_sse = kmeans(data, k, jaccard, iter)
print(f"\nJaccard:")
print(f"SSE = {jaccard_sse}")
print("\n------------------------------------")


SSE Comparison:

Euclidean:
SSE = 25323851408.26738

------------------------------------

Cosine:
SSE = 687.7671018695032

------------------------------------

Jaccard:
SSE = 3729.475436757144

------------------------------------


## Q2

In [ ]:
def compute_accuracy(true_labels, predicted_clusters):
    """
    Computes the accuracy of clustering by mapping clusters to true labels.

    Parameters:
    - true_labels: Array of true class labels.
    - predicted_clusters: Array of predicted cluster indices.

    Returns:
    - accuracy: Accuracy of the clustering.
    """
    # Get the unique clusters and true labels
    unique_clusters = np.unique(predicted_clusters)

    # Create a mapping from cluster to the majority class
    cluster_to_label = {}

    for cluster in unique_clusters:
        # Find the majority class in each cluster
        cluster_points = true_labels[predicted_clusters == cluster]
        if len(cluster_points) > 0:
            majority_label = np.bincount(cluster_points).argmax()
            cluster_to_label[cluster] = majority_label

    # Map the predicted clusters to their corresponding labels
    predicted_labels = np.array([cluster_to_label[cluster] for cluster in predicted_clusters])
    # Compute the accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    return accuracy

In [113]:
print("\nAccuracy:")
print("Euclidian = ",np.round(compute_accuracy(labels.flatten(), euclidean_clusters.astype(int))*100,2),"%")
print("Cosine = ",np.round(compute_accuracy(labels.flatten(), cosine_clusters.astype(int))*100,2),"%")
print("Jaccard = ",np.round(compute_accuracy(labels.flatten(), jaccard_clusters.astype(int))*100,2),"%")


Accuracy:
Euclidian =  60.21 %
Cosine =  58.34 %
Jaccard =  60.38 %


## Q3 & Q4

### Stop criteria: No change in centroids

In [116]:
def kmeans_tolerance(data, k, distance, tolerance_limit = 0.0001, max_iter=500):
    '''k-means algo with stop criteria: no change in centroids'''
    n, m = data.shape
    centroids = data[np.random.choice(n, k, replace=False)]
    clusters = np.zeros(n)
    
    for iter in range(max_iter):
        for i, point in enumerate(data):
            distances = [distance(point, centroid) for centroid in centroids]
            clusters[i] = np.argmin(distances)
        
        new_centroids = np.array([data[clusters == j].mean(axis=0) for j in range(k)])
        if np.all(np.abs(new_centroids - centroids) < tolerance_limit):
            break
        centroids = new_centroids
        
    sse = sum([distance(data[i], centroids[int(c)])**2 for i, c in enumerate(clusters)])
    
    return clusters, centroids, sse, iter + 1

In [120]:
k = len(np.unique(labels)) # 10


print("\nStop Criteria: No change in centroids")

go = time.time()
euclidean_clusters, euclidean_centroids, euclidean_sse, iters = kmeans_tolerance(data, k, euclidean)
finish = time.time()
print(f"\nEuclidean:")
print(f"SSE = {euclidean_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
cosine_clusters, cosine_centroids, cosine_sse, iters = kmeans_tolerance(data, k, cosine)
finish = time.time()
print(f"\nCosine:")
print(f"SSE = {cosine_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
jaccard_clusters, jaccard_centroids, jaccard_sse, iters = kmeans_tolerance(data, k, jaccard)
finish = time.time()
print(f"\nJaccard:")
print(f"SSE = {jaccard_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")


Stop Criteria: No change in centroids

Euclidean:
SSE = 25477833806.63314
Iterations =  80
Time =  36.25173735618591

------------------------------------

Cosine:
SSE = 682.0518876846062
Iterations =  70
Time =  51.39323091506958

------------------------------------

Jaccard:
SSE = 3661.2799213191965
Iterations =  74
Time =  71.9665458202362

------------------------------------


### Stop Criteria: Increase in SSE

In [121]:
def kmeans_inc_sse(data, k, distance, max_iter=500):
    '''k-means algo with stop criteria: no change in centroids'''
    n, m = data.shape
    centroids = data[np.random.choice(n, k, replace=False)]
    clusters = np.zeros(n)
    
    for iter in range(max_iter):
        for i, point in enumerate(data):
            distances = [distance(point, centroid) for centroid in centroids]
            clusters[i] = np.argmin(distances)
        
        sse = sum([distance(data[i], centroids[int(c)])**2 for i, c in enumerate(clusters)])
        new_centroids = np.array([data[clusters == j].mean(axis=0) for j in range(k)])
        new_sse = sum([distance(data[i], new_centroids[int(c)])**2 for i, c in enumerate(clusters)])
        if new_sse > sse:
            break
        centroids = new_centroids
            
    return clusters, centroids, sse, iter + 1

In [ ]:
k = len(np.unique(labels)) # 10


print("\nStop Criteria: Increase in SSE")

go = time.time()
euclidean_clusters, euclidean_centroids, euclidean_sse, iters = kmeans_inc_sse(data, k, euclidean)
finish = time.time()
print(f"\nEuclidean:")
print(f"SSE = {euclidean_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
cosine_clusters, cosine_centroids, cosine_sse, iters = kmeans_inc_sse(data, k, cosine)
finish = time.time()
print(f"\nCosine:")
print(f"SSE = {cosine_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
jaccard_clusters, jaccard_centroids, jaccard_sse, iters = kmeans_inc_sse(data, k, jaccard)
finish = time.time()
print(f"\nJaccard:")
print(f"SSE = {jaccard_sse}")
print("Iterations = ", iters)
print("Time = ", finish-go)
print("\n------------------------------------")


Stop Criteria: No change in centroids

Euclidean:
SSE = 25407782883.540108
Iterations =  500
Time =  315.3308460712433

------------------------------------

Cosine:
SSE = 687.012736092078
Iterations =  17
Time =  29.042181491851807

------------------------------------

Jaccard:
SSE = 3994.5076660085647
Iterations =  1
Time =  0.9720587730407715

------------------------------------


### Stop criteria: Max Iterations = 500

In [123]:
def kmeans_max_iter(data, k, distance, max_iter=500):
    '''k-means algo'''
    n, m = data.shape
    centroids = data[np.random.choice(n, k, replace=False)]
    clusters = np.zeros(n)
    
    for dummy in range(max_iter):
        for i, point in enumerate(data):
            distances = [distance(point, centroid) for centroid in centroids]
            clusters[i] = np.argmin(distances)
        
        centroids = np.array([data[clusters == j].mean(axis=0) for j in range(k)])
        
    sse = sum([distance(data[i], centroids[int(c)])**2 for i, c in enumerate(clusters)])
    
    return clusters, centroids, sse

In [124]:
k = len(np.unique(labels)) # 10


print("\nStop Criteria: Max Iter = 500")

go = time.time()
euclidean_clusters, euclidean_centroids, euclidean_sse = kmeans_max_iter(data, k, euclidean)
finish = time.time()
print(f"\nEuclidean:")
print(f"SSE = {euclidean_sse}")
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
cosine_clusters, cosine_centroids, cosine_sse = kmeans_max_iter(data, k, cosine)
finish = time.time()
print(f"\nCosine:")
print(f"SSE = {cosine_sse}")
print("Time = ", finish-go)
print("\n------------------------------------")

go = time.time()
jaccard_clusters, jaccard_centroids, jaccard_sse= kmeans_max_iter(data, k, jaccard)
finish = time.time()
print(f"\nJaccard:")
print(f"SSE = {jaccard_sse}")
print("Time = ", finish-go)
print("\n------------------------------------")


Stop Criteria: No change in centroids

Euclidean:
SSE = 25455066291.91364
Time =  242.48348307609558

------------------------------------

Cosine:
SSE = 684.2583124399049
Time =  378.21709537506104

------------------------------------

Jaccard:
SSE = 3660.4583924193375
Time =  793.3880848884583

------------------------------------
